In [ ]:
#TWITTER API REQUEST TOKEN KEY

ACCESS_TOKEN="159538314-SHIopOl7hBZ9hL9drK1Z9AGd61byd7Kh72ffFcQp"
ACCESS_TOKEN_SECRET="Ia16rQix3ay3iNWfaSdltasxHCTKykmsbXIjN7M6UMEkl"
CONSUMER_KEY="HWvr1EWXLKmfveEeBzK0ck5kA"
CONSUMER_SECRET="9iJGbUcuYIrPbxWpjLPBDSancWwCWcmjaR2aFIEhtILfZvQRzk"

In [ ]:
#Import Modules and Function

from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import numpy as np

In [ ]:
#TWITTER AUTHENTICATION
class TwitterAuthenticator():
    
    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
        return auth

In [ ]:
# FUNCTION to Convert Tweets to Pandas DF
class TweetAnalyzer():
    
    def tweets_to_data_frame(self,tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['len']= np.array([len(tweet.text) for tweet in tweets])
        
        return df

In [ ]:
# CLASS to Define function to crawl tweets based on Hash_tag
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [ ]:
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [ ]:
class TwitterClient():
    def __init__(self,twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user
    
    def get_user_timeline_tweets(self, num_tweets):
        tweets=[]
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_home_timeline_tweets(self,num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).item(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets
    
    def get_twitter_client_api(self):
        return self.twitter_client

In [ ]:
## Input the hash_tag list to crawl the tweets real-time
hash_tag_list=["bitcoin","cryptocurrecy"]
fetched_tweets_filename="tweets.txt"

twitter_streamer= TwitterStreamer()
twitter_streamer.stream_tweets(fetched_tweets_filename,hash_tag_list)

In [ ]:
## Input page you want to crawl from his/her timelines and Numbers

twitter_client = TwitterClient('Vitalik Non-giver of Ether')
tweet_analyzer = TweetAnalyzer()
twitter_TL = twitter_client.get_user_timeline_tweets(50)
df = tweet_analyzer.tweets_to_data_frame(twitter_TL)
print(df.head)
df.to_csv("tweets_df.csv", encoding='utf-8', index=False)